In [ ]:
from sdv.metadata import SingleTableMetadata

metadata = SingleTableMetadata()

Create metadata:

In [ ]:
import pandas as pd
df = pd.read_csv('C:\workdir\creditcard.csv')

metadata.detect_from_dataframe(data=df)
metadata.validate()

skálázás:

In [ ]:
from sklearn.preprocessing import RobustScaler

scaler = RobustScaler()

df['amount_scaled'] = scaler.fit_transform(df['Amount'].values.reshape(-1,1))
df['time_scaled'] = scaler.fit_transform(df['Time'].values.reshape(-1,1))

df.drop(['Time','Amount'], axis=1, inplace=True)
df.head()

Conditions:

In [ ]:
from sdv.sampling import Condition

frauds = Condition(
    num_rows=25000,
    column_values={'Class': 1}
)

non_frauds = Condition(
    num_rows=25000,
    column_values={'Class': 0}
)

synthetic data generation: CTGAN

In [ ]:
from sdv.single_table import CTGANSynthesizer

ctgan_synthesizer = CTGANSynthesizer(metadata)
ctgan_synthesizer.fit(df)

ctgan_df = ctgan_synthesizer.sample_from_conditions(conditions=[frauds,non_frauds])

TVAE:

In [ ]:
from sdv.single_table import TVAESynthesizer

tvae_synthesizer = TVAESynthesizer(metadata)
tvae_synthesizer.fit(df)

tvae_df = tvae_synthesizer.sample_from_conditions(conditions=[frauds,non_frauds])

Evaluate the two synthetic dataframes:

In [ ]:
from sdv.evaluation.single_table import evaluate_quality

quality_report_ctgan = evaluate_quality(
    real_data=df,
    synthetic_data=ctgan_df,
    metadata=metadata)

In [ ]:
quality_report_tvae = evaluate_quality(
    real_data=df,
    synthetic_data=tvae_df,
    metadata=metadata)

In [ ]:
from sdv.evaluation.single_table import get_column_plot

fig = get_column_plot(
    real_data=df,
    synthetic_data=ctgan_df,
    column_name='Class',
    metadata=metadata
)
    
fig.show()

In [ ]:
from sdv.evaluation.single_table import get_column_plot

fig = get_column_plot(
    real_data=df,
    synthetic_data=tvae_df,
    column_name='Class',
    metadata=metadata
)
    
fig.show()